In [1]:
import persistence_homology as ph

In [2]:
import csv

In [3]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output

In [4]:
folder_path = './lung_segmentations/segmentation-10'
edges_path = '/edges.csv'
vertices_path = '/vertices.csv'

In [5]:
vertices = read_csv(folder_path + vertices_path)
edges = read_csv(folder_path + edges_path)

In [6]:
direction = [0,0,1]

In [7]:
points = [[vertex, [0,0,0]] for vertex in vertices]
pre_edges = edges
pre_formatted_edges = []
for edge in pre_edges:
    x,y = edge
    pre_formatted_edges.append([[x, y], [1,1,1]])
pre_vertices = ph.append_height_vertices(direction, points)
vertices = ph.format_vertices(pre_vertices)
edges = ph.format_edges(vertices, pre_formatted_edges)
pre_graph = ph.process_graph(vertices, edges, direction)
graph = pre_graph['signed_graph']
original_to_new_indices = pre_graph['index_translation']
new_to_original_indices = {v: k for k, v in original_to_new_indices.items()}
filtration = ph.group_events_by_height(graph[0], graph[1])


# Filtration Algorithm Start

In [8]:
uf = ph.UnionFind(graph[0])

In [14]:
# Horizontal Step
def horizontal_step(edges: list):
    for edge in edges:
        x ,y = edge['vertices']
        uf.union(x, y)
        
def vertical_step(edges: list, components: dict, mergers: dict):
    for edge in edges:
        x ,y = edge['vertices']
        roots = uf.union(x, y)
        rootX = roots[x]['root']
        rootY = roots[y]['root']
        rankX = roots[x]['rank']
        rankY = roots[y]['rank']
        if rootX != rootY:
            if rootX in components and rootY in components:
                if rankY > rankX:
                    mergers[rootY]=max(edge['height'])
                    del components[rootY]
                elif rankX > rankY:
                    mergers[rootX]=max(edge['height'])
                    del components[rootX]
                else:
                    mergers[rootY]=max(edge['height'])
                    del components[rootY]
                    
def compute_components(vertices, old_components):
    components = old_components
    for vertex in vertices:
        node = vertex['new_index']
        root = uf.find(node)  # This finds the representative of the component containing 'node'
        if root not in components:
            components[root] = []
        if node not in components[root]:
            components[root].append(node)
    return components

def compute_new_births(vertices):
    new_components = []
    for vertex in vertices:
        node = vertex['new_index']
        root = uf.find(node)  # This finds the representative of the component containing 'node'
        if root == node:
            new_components.append(vertex)
    return new_components

def compute_intervals(births, mergers):
    """birth {'coordinates': [284.0, 315.0, 93.5], 'original_index': 18104, 'new_index': 0, 'height': 93.5, 'sign': 0}
    merger 5 98.5"""
    intervals = []
    for birth in births:
        left_bound = birth['height']
        right_bound = -1
        index = birth['new_index']
        if index in mergers.keys():
            right_bound = mergers[index]
        intervals.append([left_bound, right_bound])
    return intervals
    
def length_of_interval(interval):
    if interval[1] == -1:
        return -1
    else:
        return interval[1] - interval[0]

In [10]:
total_components = {}
mergers = {}
total_vertices = []
births = []
for height, stage in filtration.items():
    horizontal_edges = stage['horizontal_edges']
    vertical_edges = stage['vertical_edges']
    vertices = stage['points']

    # We join the horizontal edges
    horizontal_step(horizontal_edges)
    
    # We join the vertical edges
    vertical_step(vertical_edges, total_components, mergers)
    total_vertices.extend(vertices)

    # We compute the new components
    current_components = compute_components(total_vertices, total_components)
    total_components = current_components

    # We compute the horizontal connected components at a given stage, the unmerged are new components
    new_births = compute_new_births(vertices)
    births.extend(new_births)

In [12]:
intervals = compute_intervals(births, mergers)

In [15]:
def compute_largest_bar(intervals):
    largest_bar = 0
    for interval in intervals:
        if length_of_interval(interval) > largest_bar:
            largest_bar = length_of_interval(interval)
    return largest_bar

In [16]:
print(compute_largest_bar(intervals))

56.0
